In [1]:
# Groupe H :
# Nom - numéro d'étudiant
# 
# NGUYEN Huu Khang - 21506865
# 
# NGUYEN Hoai Nam -
# 
# NGUYEN Tran Tuan Nam -
#
# TRAN Thi Tra My - 21511002
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Load file
data = pd.read_csv('Dataset/claim_extraction_18_10_2019_annotated.csv', sep=',')

data2 = data.copy()

In [155]:
import time

#Import outil sklearn
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# Sickit learn met régulièrement à jour des versions et indique des futurs warnings
# Ces deux lignes permettent de ne pas les afficher
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

#Import outil nltk pour traitement
import nltk
import unicodedata
import contractions
import inflect
#nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#pickle
import pickle


In [112]:
#initialize data3 
data3 = data.copy()

#Pré-traitement choisis : 

#On supprime les colonnes qui contiennent beaucoup des valeurs null et qui ne servent pas pour notre analyse

data3 = data3.drop('claimReview_author', 1)#manque beaucoup de valeur
data3 = data3.drop('claimReview_author_name', 1)# celui ci baisse la precision
data3 = data3.drop('creativeWork_author_sameAs', 1)#manque beaucoup de valeur
data3 = data3.drop('claimReview_author_url', 1)
data3 = data3.drop('claimReview_source', 1)#les données qui répetè beaucoup, celui ci baisse la précision
data3 = data3.drop('extra_entities_author', 1)#rend le classification très lourd
data3 = data3.drop('extra_entities_body', 1)#rend le classification très lourd
data3 = data3.drop('extra_entities_claimReview_claimReviewed', 1)#rend le classification très lourd
data3 = data3.drop('extra_entities_keywords', 1)#rend le classification très lourd
data3 = data3.drop('extra_refered_links',1) #ne servent pas pour l'analyse et baisse la précision
#data3 = data3.drop('rating_bestRating', 1)
#data3 = data3.drop('rating_ratingValue', 1)
#data3 = data3.drop('rating_worstRating', 1)

print(data3.shape)
display(data3.info())

######Stop words
#initialize stop words type
stop_words = stopwords.words('english')

######Pre traiter le données du texte
def pretraiter(text):  
    if(isinstance(text, str) and text ) :
        #Remove contractions
        text = contractions.fix(text)

        #remove URL
        text = re.sub(r"http\S+", "", text)
        
        #remove photo url
        text = re.sub(r"pic.\S+", "", text)

        # Tokenizing
        tokenizedText = word_tokenize(text)
        
        #Remove non-ASCII characters from list of tokenized words
        new_words = []
        for word in tokenizedText:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        tokenizeText = new_words

        # Put all words in lowercase
        tokenizedText = [word.lower() for word in tokenizedText]

        # Delete ponctuations
        tokenizedText = [word for word in tokenizedText if word.isalpha()]

        #remove stop_word
        tokenizedText = [word for word in tokenizedText if not word in stop_words]
        
        # Converting numbers
        p = inflect.engine()
        newWords = []
        for word in tokenizedText:
            if word.isdigit():
                newWords.append(p.number_to_words(word))
            else:
                newWords.append(word)
        tokenizedText = newWords

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokenizedText = [lemmatizer.lemmatize(word, pos = 'v') for word in tokenizedText]

        # Turning back tokens into a string
        text = "".join([" " + i for i in tokenizedText]).strip()
        return text
    
    
##### Prétraiter alternateName
def pretraiter_alternatename (text) :
    if(isinstance(text, str) and text ) :
        # Tokenizing
        tokenizedText = word_tokenize(text)
        
        # Put all words in lowercase
        tokenizedText = [word.lower() for word in tokenizedText]
        
        if tokenizedText[0] == 'true' :
            return 1
        else :
            return 0
        
    else :
        return 0
    
##### Supression les liens et remplace par son domaine
def pretraiter_refered_links (text) :
    if(isinstance(text, str) and text ) :
        url_list = []
        list = text.strip().split (",")
        for element in list :
            m = re.search('https?://([A-Za-z_0-9.-]+).*', element)
            if m:
                url_list.append(m.group(1))
        return "".join(["," + i for i in url_list]).strip()
    else :
        return "Inconnu"
    
    


(39218, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39218 entries, 0 to 39217
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  39218 non-null  int64  
 1   claimReview_claimReviewed   39216 non-null  object 
 2   claimReview_datePublished   37913 non-null  object 
 3   claimReview_url             39218 non-null  object 
 4   creativeWork_author_name    17481 non-null  object 
 5   creativeWork_datePublished  17547 non-null  object 
 6   extra_body                  38866 non-null  object 
 7   extra_tags                  30239 non-null  object 
 8   extra_title                 38869 non-null  object 
 9   rating_alternateName        38964 non-null  object 
 10  rating_bestRating           3432 non-null   float64
 11  rating_ratingValue          3432 non-null   float64
 12  rating_worstRating          3164 non-null   float64
dtypes: float64(3), int6

None

In [113]:
#Exécution pretraitement
data3['extra_body'] = data['extra_body'].apply(lambda x: pretraiter(x))
data3['claimReview_claimReviewed'] = data['claimReview_claimReviewed'].apply(lambda x: pretraiter(x))
data3['rating_alternateName'] = data['rating_alternateName'].apply(lambda x: pretraiter_alternatename(x))
#data3['extra_refered_links'] = data['extra_refered_links'].apply(lambda x: pretraiter_refered_links(x))

In [114]:
#Prétraitement avec les fonctions existants
data3 = data3[data3['extra_body'].notna()]
data3 = data3[data3['claimReview_claimReviewed'].notna()]
data3['creativeWork_author_name'] = data3['creativeWork_author_name'].fillna('Inconnu')
data3['extra_tags'] = data3['extra_tags'].fillna('Inconnu')
data3['claimReview_datePublished'] = data3['claimReview_datePublished'].fillna('Inconnu')
data3['creativeWork_datePublished'] = data3['creativeWork_datePublished'].fillna('Inconnu')
data3['rating_bestRating'] = data3.replace(np.NaN, data3['rating_bestRating'].mean())
data3['rating_ratingValue'] = data3.replace(np.NaN, data3['rating_ratingValue'].mean())
data3['rating_worstRating'] = data3.replace(np.NaN, data3['rating_worstRating'].mean())

In [124]:
le = preprocessing.LabelEncoder() #transformer les valeurs str en valeurs numérique
le_data = data3.apply(le.fit_transform)
array = le_data.values
X = array[:,0:8]
y = array[:,9]

# Vectorisation avec TF-IDF
# In[ ]:

#vectorizer = TfidfVectorizer()
#vectors = vectorizer.fit_transform(X)

#X = vectors.toarray()


validation_size=0.3 #30% du jeu de données pour le test
testsize= 1-validation_size
seed=30
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=validation_size,random_state=seed,test_size=testsize)

conf = confusion_matrix(y_test, result)
print ('\n matrice de confusion \n',conf)
print ('\n',classification_report(y_test, result))

In [158]:
#classifications NB
clf = GaussianNB()
clf.fit(X_train, y_train)
result = clf.predict(X_test)
print('\nNB accuracy :', accuracy_score(result, y_test),'\n')

#classification KNeighbor
clf=KNeighborsClassifier(n_neighbors=4)
clf.fit(X, y)
result = clf.predict(X_test)
print('\nKNN accuracy :', accuracy_score(result, y_test),'\n')

#classification DecisionTreeClassifier
clf=DecisionTreeClassifier(random_state=40)
clf.fit(X_train, y_train)
result = clf.predict(X_test)
print('\nCART accuracy :', accuracy_score(result, y_test),'\n')



NB accuracy : 0.887373644550634 


KNN accuracy : 0.9421062304723397 


CART accuracy : 0.9078845800404337 



In [126]:
#Classifieurs avec les données par défauts
seed = 10
scoring = 'accuracy'
models = []
models.append(('LR', LogisticRegression(solver='lbfgs')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC(gamma='auto'))) #semble ne finisse jamais

results = []
names = []
for name,model in models:
    kfold = KFold(n_splits=10, shuffle = True, random_state=seed)
    start_time = time.time()
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring = scoring)
    #pour avoir les paramètres utilisés dans le modèle enlever commentaire ligne suivante
    #print (model.get_params())
    print ("Time pour",name," ",time.time() - start_time)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) 

Time pour LR   1.4968421459197998
LR: 0.882179 (0.002814)
Time pour KNN   2.7070658206939697
KNN: 0.925664 (0.002792)
Time pour CART   2.4269890785217285
CART: 0.907498 (0.004444)
Time pour NB   0.17253684997558594
NB: 0.886296 (0.002693)


In [ ]:
#Visualiser le résultat
fig = plt.figure()
fig.suptitle('Comparaison des algorithmes')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)

In [142]:
#Gris search
#Classifieur et ses parametres
classifiers = {
    'LogisticRegression' : LogisticRegression(solver='lbfgs'),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
}

parameters = {
    'LogisticRegression': [
        {'penalty': ['l2']}, {'C': [0.001,0.01,0.1,1,10,100,1000]}
    ],
    'KNeighborsClassifier' : [
        {'n_neighbors': list(range(1,15))}, {'metric': ['minkowski','euclidean','manhattan']}
    ],
    'DecisionTreeClassifier' : [
        {'max_depth': [1,2,3,4,5,6,7,8,9,10]}, {'criterion': ['gini', 'entropy']}, {'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10]}
    ],
}

In [143]:
class Result:
    def __init__(self,name, score, parameters):
        self.name = name
        self.parameters = parameters
        self.score = score

    def __repr__(self):
        return repr((self.name, self.score, self.parameters))
    
results = []
for key, value in classifiers.items():
    gd_sr = GridSearchCV(estimator = value, param_grid = parameters[key], scoring = "accuracy", cv = 5, n_jobs = 1, iid = True)
    gd_sr.fit(X_train, y_train)
    result = Result(key, gd_sr.best_score_, gd_sr.best_estimator_)
    results.append(result)

results = sorted(results, key = lambda result: result.score, reverse = True)

print("Results from best to worst: \n")
for result in results:
    print ("Classifier: ", result.name,
    " with score %0.2f " %result.score, 
    "avec ", result.parameters,'\n')


Results from best to worst: 

Classifier:  DecisionTreeClassifier  with score 0.94  avec  DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best') 

Classifier:  KNeighborsClassifier  with score 0.92  avec  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform') 

Classifier:  LogisticRegression  with score 0.88  avec  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='aut

In [159]:
# Utilisation d'une pipeline pour sauvegarder le meilleur modèle

print ('Création du pipeline \n')
pipeline = Pipeline([('scl', StandardScaler()),
 ('pca', PCA(n_components=2)),
 ('clf', DecisionTreeClassifier(random_state=40))])

pipeline.fit(X_train, y_train)
result = pipeline.predict(X_test)

print('\n accuracy:',accuracy_score(result, y_test),'\n')

matrix = confusion_matrix(y_test, result)
print ('\nMatrice de confusion: \n', matrix, "\n")

print ('\n', classification_report(y_test, result), "\n")

Création du pipeline 


 accuracy: 0.8165043190589965 


Matrice de confusion: 
 [[21449  2574]
 [ 2418   764]] 


               precision    recall  f1-score   support

           0       0.90      0.89      0.90     24023
           1       0.23      0.24      0.23      3182

    accuracy                           0.82     27205
   macro avg       0.56      0.57      0.57     27205
weighted avg       0.82      0.82      0.82     27205
 



In [162]:
#Sauvegarder le modèle choisi

clf=DecisionTreeClassifier(random_state=40)
clf.fit(X_train, y_train)

filename = 'pkl_modelNB.sav'
pickle.dump(clf, open(filename, 'wb'))

clf_loaded = pickle.load(open(filename, 'rb'))
print ('Modèle chargé',clf_loaded,'\n')
result = clf_loaded.predict(X_test)

print('\n accuracy:\n')
print (accuracy_score(result, y_test),'\n')

conf = confusion_matrix(y_test, result)
print ('\n matrice de confusion \n',conf)
print ('\n',classification_report(y_test, result))


pickle.dump(pipeline, open('groupeH.pkl', 'wb'))
clf_loaded = pickle.load(open('groupeH.pkl', 'rb'))

Modèle chargé DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=40, splitter='best') 


 accuracy:

0.9078845800404337 


 matrice de confusion 
 [[22710  1313]
 [ 1193  1989]]

               precision    recall  f1-score   support

           0       0.95      0.95      0.95     24023
           1       0.60      0.63      0.61      3182

    accuracy                           0.91     27205
   macro avg       0.78      0.79      0.78     27205
weighted avg       0.91      0.91      0.91     27205



In [74]:
#data3.to_csv('clean_data_fact-checking.csv', sep=';', index=False)

In [83]:
print(data['extra_refered_links'][0])

https://t.co/Oo5Q56ALAu,https://twitter.com/ianbremmer/status/1180501727960866816?ref_src=twsrc%5Etfw,https://www.instagram.com/p/B3H7aF7hHAa/,https://www.cbsnews.com/news/sauli-niinisto-and-trump-press-conference-president-finland-receives-praise-and-memes-after-white-house-meeting/,https://twitter.com/hashtag/TrumpMeltdown?src=hash&ref_src=twsrc%5Etfw,https://t.co/Ko4o4mxVLN,https://twitter.com/DogsHateBoots/status/1179476215327010816?ref_src=twsrc%5Etfw
